### SETTING THE ENV

In [ ]:
log_file="/content/sarigama_ai/logs/logs.txt"
![ -f "$log_file" ] && > "$log_file"
!mkdir -p /content/sarigama_ai/logs && touch $log_file && date > $log_file
!sudo apt update -yqq >> "$log_file" 2>&1 && sudo apt install ffmpeg -yqq >> "$log_file" 2>&1 && pip install pytube spleeter pydub mutagen yt-dlp -q >> "$log_file" 2>&1
!grep -q "ERROR" "$log_file" && echo "Something Went Wrong, Try To Run This Cell Again or Check the Log File ... " || echo "All Set To Go ... "

### LOADING THE BATTERIES 

In [ ]:
import os
import time
import shutil
import subprocess
import spleeter
import ffmpeg
import zipfile
import yt_dlp
from IPython.display import Audio
from pytube import YouTube
from pydub import AudioSegment
from google.colab import files
from mutagen.mp3 import MP3
print("Batteries Loaded ! ")

### CREATING THE PROJECT

In [ ]:
def create_project_directory():
    root_dir = "/content/sarigama_ai"
    project_name = input("\nEnter Your Project Name :  ")
    project_dir = os.path.join(root_dir, project_name)
    input_dir = os.path.join(project_dir, "input_data")
    output_dir = os.path.join(project_dir, "output_data")

    for directory in [project_dir, input_dir, output_dir]:
        if not os.path.exists(directory):
            os.makedirs(directory)
            print(f"\nCreated Directory {directory}")
    print("\nProject Structure Generated ...\n ")
    return project_name, input_dir, output_dir
pro_name,ip_dir, op_dir = create_project_directory()

### OPTION 1 : DOWNLOAD FILE

In [ ]:
def download_audio_file(res_link, ip_dir):
    start_time = time.time()
    
    try:
        if "youtube.com" in res_link:
            print("\nJust a Moment, We're Fetching the Audio File for You from YouTube ...")
            ydl_opts = {'format': 'bestaudio/best', 'outtmpl': os.path.join(ip_dir, '%(title)s.%(ext)s'), 'postprocessors': [{'key': 'FFmpegExtractAudio', 'preferredcodec': 'mp3', 'preferredquality': '0'}], 'quiet': True}
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info_dict = ydl.extract_info(res_link, download=True)
                original_audio = ydl.prepare_filename(info_dict)
                
        else:
            print("\nJust a Moment, We're Downloading the Audio File for You ...")
            file_name = res_link.split("/")[-1]
            file_path = os.path.join(ip_dir, file_name)
            os.system(f'wget -q -O "{file_path}" "{res_link}"')
            original_audio = file_path

        end_time = time.time()
        total_time = round(end_time - start_time, 2)

        print(f"\nAudio File Downloaded Successfully ... \n\nTotal Time Taken: {total_time} Seconds ... \n")
        return original_audio
    except Exception as e:
        print(f"\nError Occured While Downloading the Audio File ... \n\nChange the URL or Proceed with File Upload Option ... \n")
        return None

any_link = input("\nEnter The URL for the Audio To Be Downloaded :  ( Youtube | WebURL ) :  ")
oA = ".".join(download_audio_file(any_link, ip_dir).split(".")[:-1]) + ".mp3"


### OPTION 2 : FILE UPLOAD  [ uncomment the function call and run ]

In [ ]:
def upload_audio_file(ip_dir):
    start_time = time.time()
    print("\n\nSelect the Audio File To Upload :  ")
    uploaded_file = files.upload()
    file_name = list(uploaded_file.keys())[0]
    file_extension = os.path.splitext(file_name)[1]
    if file_extension not in [".mp3", ".wav"]:
        print("\nError: Please Upload an Audio File with .mp3 or .wav Extension  ! ")
        return None
    original_audio_path = os.path.join(os.getcwd(), file_name)
   
    with open(original_audio_path, "wb") as f:
        f.write(uploaded_file[file_name])
    shutil.move(original_audio_path, ip_dir)
    original_audio = os.path.join(ip_dir, file_name)
    end_time = time.time()
    total_time = end_time - start_time
    if total_time > 60:
        total_time = round(total_time / 60, 2)
        print(f"\nAudio File Uploaded Successfully ... \n\nTotal Time Taken : {total_time} Minutes ... ")
    else:
        total_time = round(total_time, 2)
        print(f"\nDone! Upload \n\nTotal Time Taken : {total_time} Seconds .. ")

    return original_audio
#oA = upload_audio_file(ip_dir)

### AUDIO INFO

In [ ]:
def audio_details(file_path):
    audio = MP3(file_path)
    audio_name = os.path.basename(file_path)
    audio_length = audio.info.length  # in seconds
    audio_size = os.path.getsize(file_path) / (1024 * 1024)  # in MB
    audio_bitrate = audio.info.bitrate / 1000  # in kbps

    if audio_length >= 60:
        audio_length = int(audio_length // 60)
        audio_length_str = f"{audio_length} minutes"
    else:
        audio_length = int(audio_length)
        audio_length_str = f"{audio_length} seconds"
    
    audio_details_dict = {
        "audio_name": audio_name,
        "duration": audio_length_str,
        "size": f"{audio_size:.2f} MB",
        "bitrate": f"{audio_bitrate:.2f} kbps"
    }
    
    print(audio_details_dict, "\n")
    return audio_details_dict


details = audio_details(oA)
Audio(oA)

### SEPARATIONS

In [ ]:
start_time = time.time()
!spleeter separate -p spleeter:4stems -o "{op_dir}" "{oA}" 
!spleeter separate -p spleeter:2stems -o "{op_dir}/" "{oA}" 
end_time = time.time()
time_taken = end_time - start_time
time_unit = "Seconds" if time_taken < 60 else "Minutes"
time_taken = time_taken / 60 if time_unit == "Minutes" else time_taken
print(f"\nTime Taken For Separations : {time_taken:.2f} {time_unit}")
os.rename(os.path.join(op_dir, os.listdir(op_dir)[0]), os.path.join(op_dir, 'separations'))

### CONVERSIONS

In [ ]:
wav_files_dir = op_dir+"/separations"
files_to_convert = [os.path.join(wav_files_dir, file_name) for file_name in os.listdir(wav_files_dir) if file_name.endswith('.wav')]
start_time = time.time()
for file in files_to_convert:
    wav_file = AudioSegment.from_wav(file)
    file_size_before = os.path.getsize(file)
    file_size_before_str = '{:.2f} MB'.format(file_size_before / (1024 * 1024))
    wav_bitrate = wav_file.frame_rate * wav_file.channels * 16 / 1000 

    pre_path = os.path.abspath(os.path.join(os.path.dirname(file), os.pardir))
    output_dir = os.path.join(pre_path, "conversions")
    os.makedirs(output_dir, exist_ok=True)

    mp3_file_path = os.path.join(output_dir, os.path.basename(file).replace('.wav', '.mp3'))
    mp3_file = wav_file.export(mp3_file_path, format='mp3', bitrate='192k')
    file_size_after = os.path.getsize(mp3_file_path)
    file_size_after_str = '{:.2f} MB'.format(file_size_after / (1024 * 1024))
    mp3_file = AudioSegment.from_mp3(mp3_file_path)
    mp3_bitrate = mp3_file.frame_rate * mp3_file.channels * mp3_file.sample_width / 1000 

    print('File: {}\nSize & Bitrate : Before Conversion : {} ({} kbps)\nSize & Bitrate : After: {} ({} kbps)\n'.format(file, file_size_before_str, int(wav_bitrate), file_size_after_str, int(mp3_bitrate)))
end_time = time.time()
time_taken = end_time - start_time
time_unit = "Seconds" if time_taken < 60 else "Minutes"
time_taken = time_taken / 60 if time_unit == "Minutes" else time_taken
print(f"\nTime Taken For Conversions : {time_taken:.2f} {time_unit}")

### LOAD MP3 TRACKS

In [ ]:
mp3_files_dir = op_dir+"/conversions"
target_files = [os.path.join(mp3_files_dir, file_name) for file_name in os.listdir(mp3_files_dir) if file_name.endswith('.mp3')]
for file_path in target_files:
    file_name = os.path.basename(file_path)
    print('\ntrack : ', file_name , "\n")
    display(Audio(file_path))

### WAV TRACKS


##### Access The Original .wav Files from Project/OutputData/Separations 
######  Not Recommended To Load Due To Larger File Sizes

### DOWNLOAD THE TRACKS 

In [ ]:
def download_audio_files(op_dir, project_title):
    file_type = "wav" if input("Enter 'w' to download WAV files or 'm' to download MP3 files: ") == "w" else "mp3"
    zip_file_name = f"{project_title}_{file_type}_tracks.zip"
    zip_file_path = os.path.join(os.path.dirname(op_dir), "downloads", zip_file_name)

    os.makedirs(os.path.dirname(zip_file_path), exist_ok=True)

    with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        for file_name in os.listdir(os.path.join(op_dir, "separations" if file_type == 'wav' else "conversions")):
            zip_file.write(os.path.join(op_dir, "separations" if file_type == 'wav' else "conversions", file_name), file_name)

    files.download(zip_file_path)

download_audio_files(op_dir, pro_name)